In [ ]:
from indy_utils import indydcp_client as client

import json
from time import sleep
import threading
import numpy as np
import jupyros as jr
import rospy

from std_msgs.msg import UInt16




########################### joystick number #############################################

# U = 1     1 = 16
# D = 2     2 = 32
# L = 4     3 = 64
# R = 8     4 = 128
# 6 = 512   5 = 256

#########################################################################################

joynum = 0



def callback(msg):
    
    global joynum
    joynum = msg.data

        
class Control():
    
    def __init__(self):
        rospy.init_node("jupyter_node")
        self.joynum_sub=jr.subscribe('/joystick_number', UInt16, callback)

    def grip(self,g):

#gripper hold
        if g == 1:
        
            indy.connect()      
            endtool_type = 0
            indy.set_endtool_do(endtool_type,1)  # val: 0(off), 1(on)
            sleep(1)
            indy.get_endtool_do(endtool_type)
            indy.disconnect()      


#gripper realease
        if g == 2:

            indy.connect()      
            endtool_type = 0
            indy.set_endtool_do(endtool_type,0)  # val: 0(off), 1(on)
            sleep(1)
            indy.get_endtool_do(endtool_type)
            indy.disconnect()      

    
    def garago(self,q):
        
        indy.connect()        
        indy.joint_move_to(q)
        indy.wait_for_move_finish()
        indy.disconnect()      






robot_ip = "192.168.1.232"  # Robot (Indy) IP
robot_name = "NRMK-Indy7"  # Robot name (Indy7)
indy = client.IndyDCPClient(robot_ip, robot_name)

con = Control()

#indy.wait_for_move_finish()


In [ ]:
from cmath import e
from concurrent.futures import thread
from multiprocessing import Process
import threading
import time
import socket

global tmp
global angle
angle = 0
tmp = []
tmp2 = []
joint6 = 0
gogo = 0

for i in range(5):
    tmp2.append(i)


        
def server():
    
        
    HOST='192.168.1.196'
    PORT=5005
    print("operate server")
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    server_socket.bind((HOST, PORT))
    server_socket.listen()
    client_socket, addr = server_socket.accept()

    print('Connected by', addr)
    global tmp

    cnt = 0
    while True:
        # tmp = 0
        data = client_socket.recv(1024)
        print('Reiceived from', addr, data.decode())
        angle = data.decode()

        for i in range(10):
            if angle != "0":
                tmp.append(angle)
                sleep(0.3)

        #print(tmp)
        #print(len(tmp))
        if len(tmp) > 4:                
            angle6 = tmp[5]
            val = float(angle6)
            val = int(val)
            print("val : ", val)
            print(type(val))

            #print(angle6)
            global joint6
            global gogo
            
            if val > 90:
                
                joint6 = -50 + (180-val)
                print(float(angle6))
                gogo = 1
                break
            
            elif val < 90:
                joint6 = -50 - val
                print(float(angle6))
                gogo = 1
                break
          

            
            


t_server = threading.Thread(target = server)


# t_angle = threading.Thread(target = angle)



t_server.start()




# t_angle.start()

In [ ]:
import rospy
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
import actionlib
from actionlib_msgs.msg import *
from geometry_msgs.msg import Pose, Point, Quaternion



class GoToPose():
    def __init__(self):

        self.goal_sent = False

    # What to do if shut down (e.g. Ctrl-C or failure)
        rospy.on_shutdown(self.shutdown)

        # Tell the action client that we want to spin a thread by default
        self.move_base = actionlib.SimpleActionClient("move_base", MoveBaseAction)
        rospy.loginfo("Wait for the action server to come up")

        # Allow up to 5 seconds for the action server to come up
        self.move_base.wait_for_server(rospy.Duration(5))

    def goto(self, pos, quat):

        # Send a goal
        self.goal_sent = True
        goal = MoveBaseGoal()
        goal.target_pose.header.frame_id = 'map'
        goal.target_pose.header.stamp = rospy.Time.now()
        goal.target_pose.pose = Pose(Point(pos['x'], pos['y'], 0.000),
                                         Quaternion(quat['r1'], quat['r2'], quat['r3'], quat['r4']))

        # Start moving
        self.move_base.send_goal(goal)

        # Allow TurtleBot up to 60 seconds to complete task
        success = self.move_base.wait_for_result(rospy.Duration(60)) 

        state = self.move_base.get_state()
        result = False

        if success and state == GoalStatus.SUCCEEDED:
                # We made it!
                result = True
        else:
            self.move_base.cancel_goal()

        self.goal_sent = False
        return result

    def shutdown(self):
        
        if joynum == 256:
            
            if self.goal_sent:
                self.move_base.cancel_goal()
                rospy.loginfo("Stop")
                rospy.sleep(1)

In [ ]:
## turtlebot position to indy ##
goal_posi = {'x': 5.72, 'y' : 1.12}

Block = 0
navigator = GoToPose()

while True:

    try:
        if joynum == 16: #press joystick number 1

        

            quaternion = {'r1' : 0.000, 'r2' : 0.000, 'r3' : 0.000, 'r4' : 1.000}
            rospy.loginfo("Go to (%s, %s) pose", goal_posi['x'], goal_posi['y'])
            success = navigator.goto(goal_posi, quaternion)

            if success:
                rospy.loginfo("Reached the desired pose")
                global Block
                Block = 1
                break;

            else:
                rospy.loginfo("Failed to reach the desired pose")

                # Sleep to give the last log messages time to be sent
                rospy.sleep(1)
                break;

    except rospy.ROSInterruptException:
        rospy.loginfo("Ctrl-C caught. Quitting")
        



In [ ]:

Level = 0 
while True:
    try:
        #go desk watching and grip
        if Block == 1:
            #Watching block postion
            A1 = [-15.31,-47.06,-47.04,4.41,-79.65,-50]
              #indy to block 
            A2 = [-19.53,-54.67,-79.65,3.12,-36.09, joint6]
            ##A2[5] = joint6
            # turn to the turtlebot
            A3 = [54.51,-56.55,-31.16,4.32,-75.68,-46.23]
            # To the turtlebot
            A4 = [64.73,-103.09,-3.67,4.72,-60.83,-78.51]
          
            sleep(0.7)
            
            print("joint6 degree : ", A2[5])
            
            con.grip(2) #release
            con.garago(A1)
    
            con.garago(A2)
            con.grip(1) #girp
            
            con.garago(A1)

            con.garago(A3)

            con.garago(A4)
            con.grip(2)
            sleep(1)

            con.garago(A3)
            
            con.garago(A1)
            
            global Level
            Level = 1
            
            break










    except: 
            if joynum == 512:
                indy.emergency_stop()

                indy.disconnect()



In [ ]:
## Turtlebot to goal position

goal_posi = {'x': 4.20, 'y' : 2.04}

navigator = GoToPose()

while True:

    try:
        if Level == 1:

            # Customize the following values so they are appropriate for your location

            quaternion = {'r1' : 0.000, 'r2' : 0.000, 'r3' : 0.000, 'r4' : 1.000}
            rospy.loginfo("Go to (%s, %s) pose", goal_posi['x'], goal_posi['y'])
            success = navigator.goto(goal_posi, quaternion)

            if success:
                rospy.loginfo("Reached the desired pose")
                sleep(1)
                Level = 2
                break;

            else:
                rospy.loginfo("Failed to reach the desired pose")

                # Sleep to give the last log messages time to be sent
                rospy.sleep(1)
                break;

    except rospy.ROSInterruptException:
        rospy.loginfo("Ctrl-C caught. Quitting")

In [ ]:
### Turtlebot to home position

goal_posi = {'x': 3.77, 'y' : 1.29}

navigator = GoToPose()

while True:


    try:
        if Level == 2:

            # Customize the following values so they are appropriate for your location

            quaternion = {'r1' : 0.000, 'r2' : 0.000, 'r3' : 0.000, 'r4' : 1.000}
            rospy.loginfo("Go to (%s, %s) pose", goal_posi['x'], goal_posi['y'])
            success = navigator.goto(goal_posi, quaternion)

            if success:
                rospy.loginfo("Reached the desired pose")
                
                
                break;

            else:
                rospy.loginfo("Failed to reach the desired pose")

                # Sleep to give the last log messages time to be sent
                rospy.sleep(1)
                break;

    except rospy.ROSInterruptException:
        rospy.loginfo("Ctrl-C caught. Quitting")
